In [1]:
!pip install --upgrade tensorflow==1.15

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.7/dist-packages (1.15.0)


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNNCell, RNN, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas_datareader as pdr

Preprocessing

In [3]:
df = pdr.get_data_yahoo('BBCA', start='2019-01-01', end='2021-4-12')

In [4]:
prices = df['Close'].values

In [5]:
train_prices, test_prices = train_test_split(prices, test_size=0.2, shuffle=False)

In [6]:
train_prices = train_prices.reshape(-1, 1)
test_prices = test_prices.reshape(-1, 1)

In [7]:
scaler = StandardScaler().fit(train_prices)
train_prices = scaler.transform(train_prices)
test_prices = scaler.transform(test_prices)

In [8]:
timestep = 64

In [9]:
train_prices.shape

(457, 1)

In [10]:
def construct_dataset(in_prices):
  features = []
  targets = []

  for i in range(timestep, in_prices.shape[0]):
    features.append(
        in_prices[i - timestep : i]
    )
    targets.append(
        in_prices[i]
    )
  features = np.array(features, 'float32')
  targets = np.array(targets, 'float32')

  return features, targets

In [11]:
x_train, y_train = construct_dataset(train_prices)
x_test, y_test = construct_dataset(test_prices)

Model

In [12]:
input_placeholder = tf.placeholder(tf.float32, [None, timestep, 1])
target_placeholder = tf.placeholder(tf.float32, [None, 1])

In [13]:
cell = SimpleRNNCell(64, activation='relu')

rnn_layer = RNN(cell, dtype=tf.float32)
output_layer = Dense(1)

In [14]:
out_tensor = rnn_layer(input_placeholder)
out_tensor = output_layer(out_tensor)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train

In [15]:
# mse
loss_tensor = tf.reduce_mean(0.5 * (out_tensor - target_placeholder)**2)

# optimal rmse (root mean squared error)
rmse_tensor = tf.reduce_mean(tf.sqrt(0.5 * (out_tensor - target_placeholder)**2))

In [16]:
optimizer = tf.train.AdamOptimizer().minimize(loss_tensor)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
saver = tf.train.Saver()

In [18]:
num_epoch = 5000

In [19]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  train_dict = {
      input_placeholder:x_train,
      target_placeholder:y_train
  }
  test_dict = {
      input_placeholder:x_test,
      target_placeholder:y_test
  }

  best_loss = float('inf')
  for epoch in range(num_epoch):
    sess.run(optimizer, feed_dict=train_dict)

    loss = sess.run(loss_tensor, feed_dict=train_dict)
    val_loss = sess.run(loss_tensor, feed_dict=test_dict)

    rmse = sess.run(rmse_tensor, feed_dict=train_dict)
    val_rmse = sess.run(rmse_tensor, feed_dict=test_dict)

    if val_loss < best_loss:
      best_loss = val_loss
      saver.save(sess, './best_model.ckpt')
      
    if epoch % 100 == 0:
      print(f'Epoch {epoch+1} - loss {loss:.4f} - rmse {rmse:.4f} - val_loss {val_loss:.4f} - val_rmse {val_rmse:.4f}')

Epoch 1 - loss 0.4095 - rmse 0.5183 - val_loss 5.8178 - val_rmse 2.3678
Epoch 101 - loss 0.0134 - rmse 0.0737 - val_loss 0.0265 - val_rmse 0.1435
Epoch 201 - loss 0.0068 - rmse 0.0572 - val_loss 0.0304 - val_rmse 0.1566
Epoch 301 - loss 0.0046 - rmse 0.0508 - val_loss 0.0601 - val_rmse 0.1907
Epoch 401 - loss 0.0036 - rmse 0.0450 - val_loss 0.0520 - val_rmse 0.1617
Epoch 501 - loss 0.0027 - rmse 0.0405 - val_loss 0.0621 - val_rmse 0.1821
Epoch 601 - loss 0.0019 - rmse 0.0332 - val_loss 0.0473 - val_rmse 0.1608
Epoch 701 - loss 0.0014 - rmse 0.0271 - val_loss 0.0580 - val_rmse 0.1815
Epoch 801 - loss 0.0012 - rmse 0.0251 - val_loss 0.0595 - val_rmse 0.1829
Epoch 901 - loss 0.0010 - rmse 0.0224 - val_loss 0.0455 - val_rmse 0.1561
Epoch 1001 - loss 0.0013 - rmse 0.0276 - val_loss 0.0510 - val_rmse 0.1706
Epoch 1101 - loss 0.0008 - rmse 0.0197 - val_loss 0.0608 - val_rmse 0.1895
Epoch 1201 - loss 0.0011 - rmse 0.0240 - val_loss 0.0449 - val_rmse 0.1573
Epoch 1301 - loss 0.0009 - rmse 0.022

In [20]:
x = prices[-64:]
x = x.reshape(-1, 1)
x = scaler.transform(x)
x = np.array([x])

In [21]:
prices[-1]

30.31999969482422

In [22]:
with tf.Session() as sess:
  saver.restore(sess, './best_model.ckpt')

  pred = sess.run(out_tensor, feed_dict={input_placeholder:x})
  pred = scaler.inverse_transform(pred)
  print(f'BBCA Next Day Closing Price: {pred}')

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
BBCA Next Day Closing Price: [[30.063845]]
